# Text classification
- Bag of words feature extraction
- Training a Naive Bayes classifier
- Training a decision tree classifier
- Training a maximum entropy classifier
- Training scikit-learn classifiers
- Measuring precision and recall of a classifier
- Calculating high information words
- Combining classifiers with voting
- Classifying with multiple binary classifiers
- Training a classifier with NLTK-Trainer

## Bag of words feature extraction

- The `bag of words` model is the simplest method
- It constructs a word presence feature set from all the words of an instance
- This method doesn't care about the order of the words
- This method doesn't care about how many times a word occurs

In [2]:
def bag_of_words(words):
    return dict([(word, True) for word in words])

### How to improve the bag of word method? We can...

- Construct a set of words that we want to exclude
 - Filtering stopwords
- Including significant bigrams
 - In addition to single words, it often helps to include significant bigrams

In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def bag_of_words_not_in_set(words, badwords):
    return bag_of_words(set(words) - set(badwords))

def bag_of_non_stopwords(words, stopfile = 'english'):
    badwords = stopwords.words(stopfile)
    return bag_of_words_not_in_set(words, badwords)


bag_of_non_stopwords(word_tokenize("the quick brown fox"))

{'brown': True, 'quick': True, 'fox': True}

In [8]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

def bag_of_bigrams_words(words, score_fn = BigramAssocMeasures.chi_sq, n = 200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    return bag_of_words(words + bigrams)

bag_of_bigrams_words(word_tokenize("the quick brown fox"))

{'the': True,
 'quick': True,
 'brown': True,
 'fox': True,
 ('brown', 'fox'): True,
 ('quick', 'brown'): True,
 ('the', 'quick'): True}

## Training a Naive Bayes classifier

- Use the `Bayes theorem` to predict the probability that a given feature set belongs to a particular label.
 - The formula is:  
   P(label | features) = P(label) * P(features | label) / P(features)  
   > P(label):  
       The prior probability of the label occurring, which is the likelihood that a random feature set will have the label.  
     P(features | label):  
       The prior probability of a given feature set being classified as that label.  
     P(features):  
       The prior probability of a given feature set occurring.  
     P(label | features):  
       This tells us the probability that the given features should have that label.
- `The split_label_feats should modify.`
- During traing, the `NaiveBayesClassifier` class constructs probability distributions for each feature using an `estimator` parameter.  
 - Default estimator: nltk.probability.ELEProbDist  
  - ELE stands for Expected Likelihood Estimate
  - The formula for calculating the label probabilities for a given feature is (`c` + 0.5) / (`N` + `B` / 2)  
   > `c`: count of times a single feature occurs  
     `N`: the total number of feature outcomes observed  
     `B`: the number of bins or unique features in the feature set  
 
 - Other choices  
  - we can use any `estimator` parameter we want, and there are quite a few to choose from.
  - The only constraints are that it must inherit from `nltk.probability.ProbDistI` and its constructor must take a `bins` keyword argument.
  - For example:
   - Using LacplaceProdDist class instead, which uses the formula (`c` + 1) / (`N` + `B`)  
   ```
   from nltk.probability import LaplaceProbDist  
   nb_classifier = NaiveBayesClassifier.train(train_feats, estimator = LaplaceProbDist)  
   accuracy(nb_classifier, test_feats)
   ```

In [16]:
import collections

def label_feats_from_corpus(corp, feature_detector = bag_of_words):
    label_feats = collections.defaultdict(list)
    for label in corp.categories():
        for fileid in corp.fileids(categories = [label]):
            feats = feature_detector(corp.words(fileids = [fileid]))
            label_feats[label].append(feats)
    return label_feats

def split_label_feats(lfeats, split = 0.75):
    train_feats = list()
    test_feats = list()
    for label, feats in lfeats.items():
        cutoff = int(len(feats)* split)
        train_feats.extend([(feat, label) for feat in feats[:cutoff]])
        test_feats.extend([(feat, label) for feat in feats[cutoff:]])
    return train_feats, test_feats

In [20]:
from nltk.corpus import movie_reviews

from nltk.tokenize import word_tokenize

from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy

movie_reviews.categories()
lfeats = label_feats_from_corpus(movie_reviews)
lfeats.keys()
train_feats, test_feats = split_label_feats(lfeats, split = 0.75)
nb_classifier = NaiveBayesClassifier.train(train_feats)
print('This Naive Bayes classifier has labels: ', nb_classifier.labels())

comments = ["the plot was ludicrous", "kate winslet is accessible"]
for comment in comments:
    print('comment: ', comment)
    print('predicted category: ', nb_classifier.classify(bag_of_words(word_tokenize(comment))))

# test the accuracy
accuracy(nb_classifier, test_feats)

This Naive Bayes classifier has labels:  ['neg', 'pos']
comment:  the plot was ludicrous
predicted category:  neg
comment:  kate winslet is accessible
predicted category:  pos


0.728

In [36]:
from sklearn.naive_bayes import BernoulliNB
sk_classifier = SklearnClassifier(BernoulliNB())
sk_classifier.train(train_feats)
accuracy(sk_classifier, test_feats)

0.812

### Classification probability

In [28]:
probs = nb_classifier.prob_classify(test_feats[0][0])
probs.samples()
for key in probs.samples():
    print('prob[{}] = {:.10f}'.format(key, probs.prob(key)))

prob[neg] = 0.0000000354
prob[pos] = 0.9999999646


### Most informative features

- most_informative_features()
- show_most_informative_features()

In [30]:
nb_classifier.most_informative_features(n = 10)
nb_classifier.show_most_informative_features(n = 10)

Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


### Alternative: Manual training

In [31]:
from nltk.probability import DictionaryProbDist

label_probdist = DictionaryProbDist({'pos': 0.5, 'neg': 0.5})
true_probdist = DictionaryProbDist({True: 1})
feature_probdist = {('pos', 'yes'): true_probdist, ('neg', 'no'): true_probdist}
classifier = NaiveBayesClassifier(label_probdist, feature_probdist)

## Training a decision tree classifier

In [32]:
from nltk.classify import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier.train(train_feats,
                                             binary = True,
                                             entropy_cutoff = 0.8,
                                             depth_cutoff = 5,
                                             support_cutoff = 30)
accuracy(dt_classifier, test_feats)

0.678

### How to use the `DecisionTreeClassifier.train()` class methld?
#### Tell the classifier is a binary classifier or not
- If the classifier only chooses between two labels, it is a binary classifier
- If we have multivalued features, we will want to stick to the default `binary = False`

#### Controlling uncertainty with `entropy_cutoff`
- Entropy is the uncertainty of the outcome.  
 - As entropy approaches 1.0, uncertainty increases  
 - As entropy approaches 0.0, uncertainty decreases

- The entropy_cutoff value is used during the tree refinement process  
- If the entropy of the probability distribution of label choices in the tree is greater than the entropy_cutoff value, then the tree is refined further by creating more branches.  
- If the entropy is lower than the entropy_cutoff value, then tree refinement is halted.  
- Entropy is calculated by giving nltk.probability.entropy() a MLEProbDist value created from a FreqDist of label counts.

#### Controlling tree depth with `depth_cutoff`
- The final decision tree will never be deeper than the deepth_cutoff value  
 - Default value is 100

#### Controlling decisions with `support_cutoff`
- The suppor_cutoff values controls how many labeled feature sets are required to refine the tree
- Labeled feature sets are eliminated once they no longer provide value to the training process
- When the number of labeled feature sets is less than or equal to support_cutoff, refinement stops, at least for that section of the tree

## Training a maximum entropy classifier

- The MaxentClassifier class is aka conditional exponential classifier or logistic regression classifier.
- The maximum entropy classifier converts labeled feature sets to vectors using encoding.
- The default algorithm is `iis` (improved iterative scaling)
 - train_maxent_classifier_with_iis()
- A better algorithm is `gis` (general iterative scaling)
 - train_maxent_classifier_with_gis()
- If `megam` is istalled and we specify the `megam` algorithm, then train_maxent_classifier_with_megam() is used
- The max_iter variable specifies the maximum number of iterations to go through and update the weights.
- The min_lldelta variable specifies the minimum change in the log likelihood required to continue iteratively improving the weights.

### Megam algorithm
```
>>> me_classifier = MaxentClassifier.train(train_feats, algorithm = 'megam', trace = 0, max_iter = 10)
[Found megam: ...]
>>> accuracy(me_classifier, test_feats)
0.8679...
```
- References:
 -  https://en.wikipedia.org/wiki/Maximum_entropy_classifier
 - http://www.umiacs.umd.edu/~hal/megam/

In [33]:
from nltk.classify import MaxentClassifier
me_classifier = MaxentClassifier.train(train_feats, trace = 0, max_iter = 1, min_lldelta = 0.5)
accuracy(me_classifier, test_feats)

/usr/local/lib/python3.7/site-packages/nltk/classify/maxent.py:1392: RuntimeWarning: overflow encountered in power
  exp_nf_delta = 2 ** nf_delta
/usr/local/lib/python3.7/site-packages/nltk/classify/maxent.py:1394: RuntimeWarning: invalid value encountered in multiply
  sum1 = numpy.sum(exp_nf_delta * A, axis=0)
/usr/local/lib/python3.7/site-packages/nltk/classify/maxent.py:1395: RuntimeWarning: invalid value encountered in multiply
  sum2 = numpy.sum(nf_exp_nf_delta * A, axis=0)
/usr/local/lib/python3.7/site-packages/nltk/classify/maxent.py:1402: RuntimeWarning: invalid value encountered in true_divide
  deltas -= (ffreq_empirical - sum1) / -sum2


0.5

In [34]:
me_classifier = MaxentClassifier.train(train_feats, algorithm = 'gis',
                                       trace = 0, max_iter =  10,
                                       min_lldelta = 0.5)
accuracy(me_classifier, test_feats)

0.722

## Training a scikit-learn classifier

### How to train an SklearnClassifier class?
- Create training features
- Choose and import an sklearn algorithm
- Construct an SklearnClassifier class with the chosen algorithm
- Train the SklearnClassifier class with our traing features

The main difference with NLTK classifiers is that step3 and step4 are usually combined

In [35]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB

sk_classifier = SklearnClassifier(MultinomialNB())
sk_classifier.train(train_feats)
accuracy(sk_classifier, test_feats)

0.83

### Trainin with logistic regression

In [37]:
from sklearn.linear_model import LogisticRegression
sk_classifier = SklearnClassifier(LogisticRegression())
sk_classifier.train(train_feats)
accuracy(sk_classifier, test_feats)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.892

### Training with support vector machine
- SVC
- LinearSVC
- NuSVC
- Reference
http://scikit-learn.org/stable/modules/svm.html

In [40]:
from sklearn.svm import SVC
sk_classifier = SklearnClassifier(SVC())
sk_classifier.train(train_feats)
accuracy(sk_classifier, test_feats)

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.69

In [41]:
from sklearn.svm import LinearSVC
sk_classifier = SklearnClassifier(LinearSVC())
sk_classifier.train(train_feats)
accuracy(sk_classifier, test_feats)

0.864

In [42]:
from sklearn.svm import NuSVC
sk_classifier = SklearnClassifier(NuSVC())
sk_classifier.train(train_feats)
accuracy(sk_classifier, test_feats)

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.882

## Measuring precision and recall of a classifier

`Precision` and `recall` are two common metrics used to evaluate classifiers.

### False positives (Type I error)
- Happen when a classifier classifies a feature set with a label it shouldn't have gotten

### False negatives (Type II error)
- Happen when a classifier doesn't assign a label to a feature that should have it.

### Precision
- the lack of false positives

### Recall
- the lack of false negatives

### F-measure
- Defined as the weighted harmonic mean of precision and recall. If p is the precision, and r is the recall, the formula is: 1/(alpha/p + (1-alpha)/r)  
 - alpha is a weighing constant that defaults to 0.5


In [75]:
import collections
from nltk.metrics.scores import precision
from nltk.metrics.scores import recall
from nltk.metrics.scores import f_measure

def precision_recall(classifier, testfeats):
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
    
    for i, (feats, label) in enumerate(testfeats):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
    
    precisions = {}
    recalls = {}
    
    for label in classifier.labels():
        precisions[label] = precision(refsets[label], testsets[label])
        recalls[label] = recall(refsets[label], testsets[label])
    
    return precisions, recalls

In [76]:
nb_precisions, nb_recalls = precision_recall(nb_classifier, test_feats)

for label in ['pos', 'neg']:
    print("precision: {}, recall: {}".format(nb_precisions[label], nb_recalls[label]))

precision: 0.8988326848249028, recall: 0.924
precision: 0.9218106995884774, recall: 0.896


In [77]:
me_precisions, me_recalls = precision_recall(me_classifier, test_feats)
for label in ['pos', 'neg']:
    print("precision: {}, recall: {}".format(me_precisions[label], me_recalls[label]))

precision: 0.8992248062015504, recall: 0.928
precision: 0.9256198347107438, recall: 0.896


## Calculating high information words

A `high information` word is a word that is strongly biased towards a single classification label.
- Similar to the `show_most_informative_features()` method on both the NaiveBayesClassifier class and the MaxentClassifier class

The `low information` words are words that are common to all labels.
- It may be counter-intuitive, but eliminating these words from the training data can actually improve accuracy, recision, and recall

### How to get the high_information_words?
- The default score function is nltk.metrics.BigramAssocMeasures.chi_sq(), chich calculates the chi-square score for each word using the following parameters:
 - n_ii: the frequency of the word for the label
 - n_ix: the total frequency of the word across all labels
 - n_xi: the total frequency of all words that occurred for the label
 - n_xx: the total frequency for all words in all labels
 
- Other score functions:
 - phi_sq() phi-square
 - pmi() pointwise mutual information
 - jaccard()
 - reference: http://www.nltk.org/_modules/nltk/metrics/association.html

In [61]:
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist

def high_information_words(labelled_words, score_fn = BigramAssocMeasures.chi_sq, min_score = 5):
    word_fd = FreqDist()
    label_word_fd = ConditionalFreqDist()
    
    for label, words in labelled_words:
        for word in words:
            word_fd[word] += 1
            label_word_fd[label][word] += 1
    
    n_xx = label_word_fd.N()
    high_info_words = set()
    
    for label in label_word_fd.conditions():
        n_xi = label_word_fd[label].N()
        word_scores = collections.defaultdict(int)
    
        for word, n_ii in label_word_fd[label].items():
            n_ix = word_fd[word]
            score = score_fn(n_ii, (n_ix, n_xi), n_xx)
            word_scores[word] = score
    
        bestwords = [word for word, score in word_scores.items() if score >= min_score]
        high_info_words |= set(bestwords)

    return high_info_words

def bag_of_words_in_set(words, goodwords):
    return bag_of_words(set(words) & set(goodwords))

In [62]:
labels = movie_reviews.categories()
labeled_words = [(l, movie_reviews.words(categories=[l])) for l in labels]
high_info_words = set(high_information_words(labeled_words))
feat_det = lambda words: bag_of_words_in_set(words, high_info_words)
lfeats = label_feats_from_corpus(movie_reviews, feature_detector = feat_det)
train_feats, test_feats = split_label_feats(lfeats)

In [63]:
nb_classifier = NaiveBayesClassifier.train(train_feats)
accuracy(nb_classifier, test_feats)

0.91

In [78]:
nb_precisions, nb_recalls = precision_recall(nb_classifier, test_feats)

## Combining classifiers with voting

- One way to improve classification performance is to combine classifiers.
- The simplest way to combine multiple classifiers is to use voting, and choose whichever label gets the most votes
- It's best to have an odd number of classifiers so that there are no ties
- The individual classifiers should also use different algorithms

In [80]:
import itertools
from nltk.classify import ClassifierI
from nltk.probability import FreqDist

class MaxVoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        self._labels = sorted(set(itertools.chain(*[c.labels() for c in classifiers])))

    def labels(self):
        return self._labels
    
    def classify(self, feats):
        counts = FreqDist()
        
        for classifier in self._classifiers:
            counts[classifier.classify(feats)] += 1
        
        return counts.max()

In [81]:
mv_classifier = MaxVoteClassifier(nb_classifier, dt_classifier, me_classifier, sk_classifier)

In [82]:
mv_classifier.labels()

['neg', 'pos']

In [83]:
accuracy(mv_classifier, test_feats)

0.908

In [84]:
mv_precisions, mv_recalls = precision_recall(mv_classifier, test_feats)
for label in mv_classifier.labels():
    print('{}: precision - {}, recall - {}'.format(label, mv_precisions[label], mv_recalls[label]))

neg: precision - 0.9146341463414634, recall - 0.9
pos: precision - 0.9015748031496063, recall - 0.916


## Classifying with multiple binary classifiers

In [43]:
from nltk.corpus import reuters
len(reuters.categories())

90

In [52]:
def reuters_high_info_words(score_fn = BigramAssocMeasures.chi_sq):
    labeled_words = []
    
    for label in reuters.categories():
        labeled_words.append((label, reuters.words(categories=[label])))
    
    return high_information_words(labeled_words, score_fn = score_fn)

def reuters_train_test_feats(feature_detector = bag_of_words):
    train_feats = []
    test_feats = []
    for fileid in reuters.fileids():
        if fileid.startswith('training'):
            featlist = train_feats
        else:
            featlist = test_feats
        feats = feature_detector(reuters.words(fileid))
        labels = reuters.categories(fileid)
        featlist.append((feats, labels))
    return train_feats, test_feats

In [79]:
rwords = reuters_high_info_words()
featdet = lambda words: bag_of_words_in_set(words, rwords)
multi_train_feats, multi_test_feats = reuters_train_test_feats(featdet)

## Training a classifier with NLTK-Trainer

- https://github.com/japerk/nltk-trainer
- http://nltk-trainer.readthedocs.org/